## Learning Algorithm for multi agent Model NON LINEAR

This script presents the algorithm that uses the Non-linear model to run trajectories and improve controllers

### Learning algorithms 

#### 'LSOptimiser' and 'GradDescent'

These runs trajectories and populate the LSTD matrices before solving them for the weight vector that gives the approximation of the Q-function

#### 'PolicyMaximiser'

This takes the learned weight vector and calculates the new controller

##### Cells at the bottom of this notebook give the scripts used to run multiple updates and test the controllers, see below


### Model:

#### 'Vehicles' 

Each of the three vehicles is treated as a separate object defined by this class

#### 'Initialise'

This is a function that given an initial state vector, will instantiate three vehicles that can be used for simulations

#### 'RunEpisode'

This takes as input the three vehicles (as objects) and a controller and runs an episode of chosen length, returning the states, costs and inputs

#### 'PlotResult' 

This is used to plot the states, costs and inputs for a given trajectory simulated as above






In [1]:
import numpy as np
import pdb
import matplotlib.pyplot as plt
import time
import math


#####################################
#Define variables to be used globally:
#######################################

v0 = 25
Tau_ref = 0.4
d0 = v0*Tau_ref

############
# NOISE
#############

noisy_controller=False
##noisy_model = False

model_variance = .2

controller_variance = 0.25

##########
# COSTS
##########

#Q1, Q2, Q3 punish v1,v2,v3**2 for each vehicle
#Q4,Q5 punish the time gap for each vehicle given by: (d12 - Tau_ref * v_following)
# R1,R2,R3 punish the difference on torque from the reference (HOW to make -ve torque have lower cost)
Q1 = 0.05
Q2 = 0.05
Q3 = 0.05
Q4 = .1
Q5 = .1
R1 = 100
R2 = 100
R3 = 100


Q = np.array([[Q1, 0, 0, 0, 0],
                   [0, Q4, -Tau_ref * Q4, 0, 0],
                   [0, -Tau_ref * Q4, Q2 + Tau_ref**2 * Q4, 0, 0],
                   [0,0,0,Q5,-Tau_ref*Q5],
                   [ 0,0,0,-Tau_ref * Q5, Q3 + Tau_ref**2 * Q5 ]])


R = np.array([[R1,0,0],
                 [0,R1,0],
                 [0,0,R1]])        

disc_fact=0.99



class Vehicles():
    
    ###########################################################
    # Constants that are common to all instances (all vehicles)
    ###########################################################
    
#Define all physical constants needed for model
    Aa = 20
    Cd = 0.6
    Cr = 0.005
    rho = 1.2
    Je = 10
    Jw=3
    r_w = 0.6
    trans_ratio = 1
    efficiency = 0.9
    m = 30000
    g = 9.81

    # Create constants to make the model much simpler
    K_denominator = Jw + (m*r_w**2) + trans_ratio**2 * efficiency * Je

    K1 = r_w * trans_ratio * efficiency /K_denominator
    K2 = (0.5 * Aa * rho * Cd * r_w**2)/K_denominator
    K3 = (Cr*m*g*r_w**2)/K_denominator
    K4 = (m*g*r_w**2)/K_denominator

    # Define coefficients of f_i(d) - the approximation for the non linear equation giving reduction in drag
    omega = [-0.9379, -0.4502,-0.43735]

    epsilon = [12.896, 43.0046, 51.5027]
    
    
    alpha = 0
    
    ##########################
    # Linearisation Points:
    ##########################
    
    v0 = 25
    Tau_ref = 0.4
    d0 = v0*Tau_ref
    
    T0 = [(K2 * (1-0.01*(omega[0]*d0+epsilon[0])) * v0**2 + K3*np.cos(alpha) + K4*np.sin(alpha))/K1,
          (K2 * (1-0.01*(omega[1]*d0+epsilon[1])) * v0**2 + K3*np.cos(alpha) + K4*np.sin(alpha))/K1,
          (K2 * (1-0.01*(omega[2]*d0+epsilon[2])) * v0**2 + K3*np.cos(alpha) + K4*np.sin(alpha))/K1 ]
    
    
    # Theoretical max input that would make it stay at 35ms^-1 :
    
    T_max = (K2*35**2 + K3*np.cos(alpha) + K4*np.sin(alpha))/K1
    
    ###########
    # Equilibrium Torque Values
    #T0= [3875.49, 2825.92, 2567.12]
    #T_max = 6860


    # Define SAMPLING TIME for discretisation
    # 50ms
    Ts = 5e-2
    
    
    def __init__(self, vehicle_number, pos_initial, vel_initial):
        # vehicle_number = 0,1,2
        
        self.i = vehicle_number
        self.position = pos_initial
        self.velocity = vel_initial 
        

        
    def GetPhi(self, distance):
        # function which calculates the value of the function phi depending on vehicle number and current distance
        
        #phi = 1-f/100
        #if distance < 0:
            #raise ValueError("d shouldn't be negative...") 
            
        
        if self.i == 0:
            if distance <= 15:
                f = self.omega[self.i]* distance + self.epsilon[self.i]
            else:
                f = 0
        elif self.i == 1 or self.i == 2:
            if distance <= 80:
                f = self.omega[self.i]* distance + self.epsilon[self.i]
            else:
                f = 0
        
        phi = 1-f/100
        
        # scalar
        return phi
    
        
    def GetAcceleration(self,d_Torque,distance):
        
        # This method finds the current acceleration which is a function of:
            # Torque
            # Intervehicle distance in front (or behind for first vehicle)
            # Speed (vehicle state so not an argument)
            # alpha (ignore for now)
        
        #pdb.set_trace()
        v_dot_engine = (self.K1 * self.GetTorque(d_Torque))
        v_dot_drag = - (self.K2 * self.GetPhi(distance) * self.velocity **2) 
        v_dot_slip = - self.K3 * np.cos(self.alpha) 
        v_dot_gravity = - self.K4 * np.sin(self.alpha)
        
        #if fault:
        #    noisy_multiplier = 1.5
        #else:
        #    noisy_multiplier = 1
        
        
        v_dot = v_dot_engine + v_dot_drag + v_dot_slip + v_dot_gravity
        #v_dot = v_dot * noisy_multiplier
        
        #pdb.set_trace()

        # scalar
        return v_dot
        
    def GetTorque(self,delta_Torque):
        # returns the torque for the vehicle given an input torque on top of eq
        
        eq_Torque = self.T0[self.i]
        
        new_Torque = eq_Torque + delta_Torque
        
        #pdb.set_trace()
        
        # Limit Torque to be no greater than 6850 and vice versa in the negative - this is now done in 'getinput'
        #pdb.set_trace()
        #new_Torque = min(new_Torque, 6850)
        #new_Torque = max(new_Torque,-6850)

        # scalar
        return new_Torque
        
    def UpdateVelocity(self,Torque_input,distance):
        # This method updates the velocity of the vehicle
        
        #Velocity = old velocity + T * acc
        self.velocity += self.GetAcceleration(Torque_input,distance) * self.Ts
        
        #if noisy_model == True:
        self.velocity += model_variance * np.random.randn(1)[0]
        
        # prevent velocity ever being negative or above 38:
        
        self.velocity = max(self.velocity,0)
        self.velocity = min(self.velocity,32)
               
        #updates velocity attribute
        return
        
    def UpdatePosition(self):
        # THis method updates the position of the vehicle
        
        self.position += self.velocity * self.Ts
        
        #updates position attribute
        return
    
def GetDistance(vehiclei, vehiclei_next):
    # Returns the intervehicle distance between vehicle i and vehicle i next
    # Arguments are OBJECTS
    
    ############
     #If you want distance for first or second vehicle: enter GetDistance(vehicle1,vehicle2)
     #If you want distance for the third vehicle: enter GetDistance(vehicle2,vehicle3)
    ############
    ## Also RETURNS A FLAG IF A NEGATIVE DISTANCE IS DETECTED SO THE EPISODE CAN BE REJECTED
    #########################################################################################
                     
    pos1 = vehiclei.position
    pos2 = vehiclei_next.position
    
    dist = pos1-pos2
    
    discard_flag = False
    
    if dist <0:
        #raise ValueError('Error, negative distance')
        #print('WARNING, distance is negative - flag raised')
        #print(f'vehicle {vehiclei.i} position is: {pos1}, vehicle {vehiclei_next.i} position is {pos2} so distance is: {dist}')
        discard_flag = True
        return dist,discard_flag
    else:
        return dist,discard_flag  #scalar and boolean
    
def RunStep(vehicle1,vehicle2,vehicle3,input_Vector):
    # Carries out an update to the state space:
    # input the input_Vector as a 3x1 array
    
    Torque1 = input_Vector[0][0]
    Torque2 = input_Vector[1][0]
    Torque3 = input_Vector[2][0]
    
    d_12,discard_flag1 = GetDistance(vehicle1,vehicle2)
    d_23,discard_flag2 = GetDistance(vehicle2,vehicle3)
    
    ### Update positions (since this uses current velocity value so want to use this before it gets updated)
    
    vehicle1.UpdatePosition()
    vehicle2.UpdatePosition()
    vehicle3.UpdatePosition()
    
    ## Then update velocities since this will affect the positions
    
    vehicle1.UpdateVelocity(Torque1,d_12)
    vehicle2.UpdateVelocity(Torque2,d_12)
    vehicle3.UpdateVelocity(Torque3,d_23)
    
    
    #If GetDistance returns a flag indicating any distance was negative: set the DISCARD to true and pass this through to runepisode 
    DISCARD = False
    
    if discard_flag1 == True or discard_flag2 == True:
        DISCARD = True
    


    # updates velocity and position attributes of 3 input objects
    return DISCARD

def GetState(vehicle1,vehicle2,vehicle3):
    #returns the state vector in array form (centred around the steady state of:
    #25,10,25,10,25
    
    v1 = vehicle1.velocity-v0
    v2 = vehicle2.velocity-v0
    v3 = vehicle3.velocity-v0
    d12,flag1 = GetDistance(vehicle1,vehicle2)
    d23,flag2 = GetDistance(vehicle2,vehicle3)
    
    d12 -=d0
    d23 -=d0
    
    # 5x1 array
    return np.array([v1,d12,v2,d23,v3]).reshape(5,1)
    

def GetPolicyInput(x,K):
    # Returns an input for the system given a state
    
    inp = np.matmul(K,x).reshape(3,1)
    
    #pdb.set_trace()
    
    if noisy_controller:
        inp += controller_variance * np.random.randn(3,1)
    #pdb.set_trace()    
        
    # must apply the actuation limit here!!!!
    limited_inp1 = min(inp[0][0], 0.685)
    limited_inp1 = max(limited_inp1, -.685)
    limited_inp2 = min(inp[1][0], .685)
    limited_inp2 = max(limited_inp2, -.685)
    limited_inp3 = min(inp[2][0], .685)
    limited_inp3 = max(limited_inp3, -.685)     
    
    
    corrected = np.array([limited_inp1, limited_inp2, limited_inp3]).reshape(3,1)
    
    
    #3x1 array
    return corrected, inp

def GetCost(x, u):
    #For a given state and input, return the one step cost of this new state
    # x: 5x1 array
    # u: 3x1 array

    # x'Ex
    cost1 = np.matmul(np.matmul(x.transpose(),Q),x)

    # u'Fu
    cost2 = np.matmul(np.matmul(u.transpose(),R),u)


    #scalar
    return (cost1+cost2)[0][0]
    
    
def Initialise(initial_state_vector):
    # Instantiates the objects of the vehicles with the given state vector input into this function
    
    # NB state_vector is in the form: centred around zero
    
    #for example, [0,0,0,0,0] would indicate an initial state of the equilibrium point
    
    v1_0 = initial_state_vector[0]
    d12_0 = initial_state_vector[1]
    v2_0 = initial_state_vector[2]
    d23_0 = initial_state_vector[3]
    v3_0 = initial_state_vector[4]


    
    vehicle1 = Vehicles(0,d12_0+d23_0+2*10,  v1_0+25)
    vehicle2 = Vehicles(1,d23_0+10,        v2_0+25)
    vehicle3 = Vehicles(2,0,            v3_0+25)
    
    #returns 3 objects
    return vehicle1,vehicle2,vehicle3
    
    

def RunEpisode(ep_length,vehicle1,vehicle2,vehicle3,K):
    # Runs an episode and returns lists of all the states
    
    x = GetState(vehicle1,vehicle2,vehicle3)


    # Create vectors to store all states
    v1 = [x[0][0]]
    d12 =[x[1][0]]
    v2 = [x[2][0]]
    d23 =[x[3][0]]
    v3 = [x[4][0]]

    Costs = []
    u1 = []
    u2 = []
    u3 = []
    
    #u1_unc = []
    #u2_unc = []
    #u3_unc = []
    
    discard_episode = False

    #Run  iterations:
    for k in range(ep_length):
        
        # current state is x
        
        current_input, Uncorrected_input = GetPolicyInput(x,K)
        #pdb.set_trace()
                
        u1.append(current_input[0][0])
        u2.append(current_input[1][0])
        u3.append(current_input[2][0])
        #u1_unc.append(Uncorrected_input[0][0])
        #u2_unc.append(Uncorrected_input[1][0])
        #u3_unc.append(Uncorrected_input[2][0])
        
        
        
        actual_input = 10e4* current_input   #since we need this factor because the K we are using is much smaller so the B matrix is correct

        dist_neg = RunStep(vehicle1,vehicle2,vehicle3,actual_input) # this function carries out the update to all vehicles
        # dist_neg is True if a negative distance was encountered so the episode should be rejected
        
        #since this will occur for every step, we only want to edit the discard_ep if we get an error
        # otherwise it will only matter what the last transition is
        if dist_neg == True:
            discard_episode = True
        
        
        x = GetState(vehicle1,vehicle2,vehicle3) #update state
        
        iteration_cost = GetCost(x,current_input)
        Costs.append(iteration_cost)

        v1.append(x[0][0])
        v2.append(x[2][0])
        v3.append(x[4][0])

        d12.append(x[1][0])
        d23.append(x[3][0])

    
    #returns 5 lists of the evolution of the states and a list of costs
    ##################################################
    # NB states returned here are centred on zero
    #################################################
    return v1,v2,v3,d12,d23,Costs,u1,u2,u3,discard_episode#,u1_unc,u2_unc,u3_unc
    #if discard_episode is True, we need to reject the episode
    
    
def RecentreVel(vel_list):
    #recentres a velocity list by adding 25 to each element (since linearised around 25)
    # vel_list: a LIST of scalars (not arrays) - these are the values of a particular state, not all states

    for i,item in enumerate(vel_list):

        vel_list[i]+= v0

    return vel_list

def RecentreDist(dist_list):
    #recentres a distance list by adding 10 to each element (since linearised around 10)
    # dist_list: a LIST of scalars (not arrays) - these are the values of a particular state, not all states

    for i,item in enumerate(dist_list):

        dist_list[i]+= d0

    return dist_list

def PlotResult(v1,v2,v3,d12,d23,u1, u2, u3, figname,K_name,Cost,model_variance,save = False):
    # Takes in the extracted states for all 5 states and plots and saves the figure
    # figname, K_name must be strings
    # f_name and noise_name will be numbers


    fig,axes = plt.subplots(3,1,figsize=(20,10))
    #axes=fig.add_axes([0,0,0.9,0.9])
    axes1 = axes[0]
    axes2 = axes[1]
    axes3 = axes[2]

    X = range(len(v1))
    Y = range(len(u1))
    
    v1r = RecentreVel(v1)
    v2r = RecentreVel(v2)
    v3r = RecentreVel(v3)
    d12r = RecentreDist(d12)
    d23r = RecentreDist(d23)
    

    axes1.plot(X,v1r,'b',label='v1')
    axes1.plot(X,v2r,'r',label='v2')
    axes1.plot(X,v3r,'g',label='v3')
    axes1.set_xlabel('Time Step')
    axes1.set_ylabel('Speed')
    axes1.legend()
    
    axes1.set_title(f'{K_name}, Average Cost = {Cost}, model variance = {model_variance}')
    #axes1.set_ylim([15,34])

    axes2.plot(X,d12r,'b',label='dist_12')
    axes2.plot(X,d23r,'r',label='dist_23')
    axes2.plot(X,np.zeros((len(v1))))
    axes2.set_xlabel('Time Step')
    axes2.set_ylabel('Inter-vehicle distance')
    axes2.legend()
    #axes2.set_ylim([8,38])
    
    axes3.plot(Y,u1,'b',label='u1')
    axes3.plot(Y,u2,'r',label='u2')
    axes3.plot(Y,u3,'g',label='u3')
    axes3.set_xlabel('Time Step')
    axes3.set_ylabel('Input')
    axes3.legend()
    axes3.set_ylim([-1,1])
    
    

    if save:
        plt.savefig(f'{figname}.png')
        
def GetDiscountedCost(Co_list):
    # given an input of the costs list of a trajectory, this will return the discounted cost of the trajectory
    # ie sum(gamma^k * R_t)

    COST = 0
    for k,cost in enumerate(Co_list):
        COST += disc_fact**k *cost

    return COST 

In [2]:
#######################################################
## Learning and Improvement functions
#######################################################

def LSOptimiser(K,number_of_episodes=10,episode_length=50):
        # This function carries out the LS estimation of the action value function. It runs a certain number of iterations, then uses the LS algorithm to estimate the action value function 
        # There are 36 states and a constant term 
        #######
        # It will return two matrices: A and B: which will solve for Theta as A*Theta = B
        
        # These are the lists that contains lists of each episodes
        V1_Matrix_list = []
        V2_Matrix_list = []
        V3_Matrix_list = []
        D12_Matrix_list = []
        D23_Matrix_list = []
        
        U1_Matrix_list = []
        U2_Matrix_list = []
        U3_Matrix_list = []
        
        Costs_Matrix_list = []
        
        ############################
        # Define variance for initial states
        # Don't want initial distance to be negative
        ###################################
        sigma_v1 = 3
        sigma_v2 = 3
        sigma_v3 = 3
        sigma_d12 = 5
        sigma_d23 = 5
        
        def GetRandomInitialState():
        #returns a random initial state
            v1_in = sigma_v1 * np.random.randn(1)[0]
            v2_in = sigma_v1 * np.random.randn(1)[0]
            v3_in = sigma_v1 * np.random.randn(1)[0]
            
            d12_in = 5+sigma_d12 * np.random.randn(1)[0]
            d23_in = 5+sigma_d23 * np.random.randn(1)[0]
            
            #prevent negative initial speeds and distances:
            
            #pdb.set_trace()
            v1_in= max(v1_in,-24.9)
            v2_in= max(v2_in,-24.9)
            v3_in = max(v3_in,-24.9)
            
            d12_in = max(d12_in,-9.9)
            d23_in = max(d23_in,-9.9)
            
            
        
            return [v1_in,d12_in,v2_in,d23_in,v3_in]
        
        ten_perc = number_of_episodes/10
        
        rejected_episodes = 0
        
        for episode in range(number_of_episodes):
            
            if episode%ten_perc == 0 and episode !=0:
                print(f'...running. Episode {episode} complete, {number_of_episodes-episode} episodes remaining')

            
            # Get initial state
            X_initial_state = GetRandomInitialState()   #this is the 'centred around 0' version

            #Initialise Vehicles (or reset them)
            vehicle1,vehicle2,vehicle3 = Initialise(X_initial_state)
            
            v1,v2,v3,d12,d23,Co,u1,u2,u3,error_flag = RunEpisode(episode_length,vehicle1,vehicle2,vehicle3,K)
            
            #If flag is raised, we just need to not store the sample:
            if error_flag:
                rejected_episodes +=1
                #print('Episode Rejected!')
                continue

            
            V1_Matrix_list.append(v1)
            V2_Matrix_list.append(v2)
            V3_Matrix_list.append(v3)
            D12_Matrix_list.append(d12)
            D23_Matrix_list.append(d23)

            U1_Matrix_list.append(u1)
            U2_Matrix_list.append(u2)
            U3_Matrix_list.append(u3)

            Costs_Matrix_list.append(Co)
            
            #pdb.set_trace()
            
        # Now we have a list of lists for each of the states, inputs and costs
        
        # Each list contains p episodes
        # Each episode contains k transitions and k+1 states
        # There are therefore p*k transitions in total = N
        
        N = 0
        
        for episode in Costs_Matrix_list:
            N += len(episode)
            
        Beta_k = np.zeros((37,N))
        Beta_kplus1 = np.zeros((37,N))
        C_k = np.zeros((N)).reshape(N,1)
        
        transition_counter = 0
        
        #pdb.set_trace()
        
        print(f'...populating LS Matrices')
        
        # for each transition, we can create an equation and hence populate a column of our matrices
        
        for idx_episode,episode in enumerate(Costs_Matrix_list):
            for idx_transition,transition in enumerate(episode):
                
                v1_k = V1_Matrix_list[idx_episode][idx_transition]
                v1_knext = V1_Matrix_list[idx_episode][idx_transition+1]
                
                v2_k = V2_Matrix_list[idx_episode][idx_transition]
                v2_knext = V2_Matrix_list[idx_episode][idx_transition+1]
                
                v3_k = V3_Matrix_list[idx_episode][idx_transition]
                v3_knext = V3_Matrix_list[idx_episode][idx_transition+1]
            
                d12_k = D12_Matrix_list[idx_episode][idx_transition]
                d12_knext = D12_Matrix_list[idx_episode][idx_transition+1]
                
                d23_k = D23_Matrix_list[idx_episode][idx_transition]
                d23_knext = D23_Matrix_list[idx_episode][idx_transition+1]
                
                u1_k = U1_Matrix_list[idx_episode][idx_transition]
                u2_k = U2_Matrix_list[idx_episode][idx_transition]
                u3_k = U3_Matrix_list[idx_episode][idx_transition]
                
                x_next_vector = np.array([v1_knext, d12_knext, v2_knext, d23_knext,v3_knext]).reshape(5,1)
                input_next = np.matmul(K,x_next_vector).reshape(3,1)
                
                u1_knext = input_next[0][0]
                u2_knext = input_next[1][0]
                u3_knext = input_next[2][0]
                
                trans_cost = Costs_Matrix_list[idx_episode][idx_transition]
        
        
            ##########################################################    
            #Populate Beta_k matrix
                #
                Beta_k[0][transition_counter] = v1_k * v1_k
                #
                Beta_k[1][transition_counter] = v1_k * v2_k
                #
                Beta_k[2][transition_counter] = v1_k * v3_k
                #
                Beta_k[3][transition_counter] = v1_k * d12_k
                #
                Beta_k[4][transition_counter] = v1_k * d23_k
                #
                Beta_k[5][transition_counter] = v1_k * u1_k
                #
                Beta_k[6][transition_counter] = v1_k * u2_k
                                #
                Beta_k[7][transition_counter] = v1_k * u3_k
                #
                Beta_k[8][transition_counter] = v2_k * v2_k
                #
                Beta_k[9][transition_counter] = v2_k * v3_k
                #
                Beta_k[10][transition_counter] = v2_k * d12_k
                #
                Beta_k[11][transition_counter] = v2_k * d23_k
                #
                Beta_k[12][transition_counter] = v2_k * u1_k
                #
                Beta_k[13][transition_counter] = v2_k * u2_k
                                #
                Beta_k[14][transition_counter] = v2_k * u3_k
                #
                Beta_k[15][transition_counter] = v3_k * v3_k
                #
                Beta_k[16][transition_counter] = v3_k * d12_k
                #
                Beta_k[17][transition_counter] = v3_k * d23_k
                #
                Beta_k[18][transition_counter] = v3_k * u1_k
                #
                Beta_k[19][transition_counter] = v3_k * u2_k
                #
                Beta_k[20][transition_counter] = v3_k * u3_k
                                #
                Beta_k[21][transition_counter] = d12_k * d12_k
                #
                Beta_k[22][transition_counter] = d12_k * d23_k
                #
                Beta_k[23][transition_counter] = d12_k * u1_k
                #
                Beta_k[24][transition_counter] = d12_k * u2_k
                #
                Beta_k[25][transition_counter] = d12_k * u3_k
                #
                Beta_k[26][transition_counter] = d23_k * d23_k
                #
                Beta_k[27][transition_counter] = d23_k * u1_k
                                #
                Beta_k[28][transition_counter] = d23_k * u2_k
                #
                Beta_k[29][transition_counter] = d23_k * u3_k
                #
                Beta_k[30][transition_counter] = u1_k * u1_k
                #
                Beta_k[31][transition_counter] = u1_k * u2_k
                #
                Beta_k[32][transition_counter] = u1_k * u3_k
                #
                Beta_k[33][transition_counter] = u2_k * u2_k
                #
                Beta_k[34][transition_counter] = u2_k * u3_k
                #
                Beta_k[35][transition_counter] = u3_k * u3_k
                #
                Beta_k[36][transition_counter] = 1
                
                #############################################
                #Populate Beta_k matrix
                #
                Beta_kplus1[0][transition_counter] = v1_knext * v1_knext * disc_fact
                #
                Beta_kplus1[1][transition_counter] = v1_knext * v2_knext * disc_fact
                #
                Beta_kplus1[2][transition_counter] = v1_knext * v3_knext * disc_fact
                #
                Beta_kplus1[3][transition_counter] = v1_knext * d12_knext * disc_fact
                #
                Beta_kplus1[4][transition_counter] = v1_knext * d23_knext * disc_fact
                #
                Beta_kplus1[5][transition_counter] = v1_knext * u1_knext * disc_fact
                #
                Beta_kplus1[6][transition_counter] = v1_knext * u2_knext * disc_fact
                                #
                Beta_kplus1[7][transition_counter] = v1_knext * u3_knext * disc_fact
                #
                Beta_kplus1[8][transition_counter] = v2_knext * v2_knext * disc_fact
                #
                Beta_kplus1[9][transition_counter] = v2_knext * v3_knext * disc_fact
                #
                Beta_kplus1[10][transition_counter] = v2_knext * d12_knext * disc_fact
                #
                Beta_kplus1[11][transition_counter] = v2_knext * d23_knext * disc_fact
                #
                Beta_kplus1[12][transition_counter] = v2_knext * u1_knext * disc_fact
                #
                Beta_kplus1[13][transition_counter] = v2_knext * u2_knext * disc_fact
                                #
                Beta_kplus1[14][transition_counter] = v2_knext * u3_knext * disc_fact
                #
                Beta_kplus1[15][transition_counter] = v3_knext * v3_knext * disc_fact
                #
                Beta_kplus1[16][transition_counter] = v3_knext * d12_knext * disc_fact
                #
                Beta_kplus1[17][transition_counter] = v3_knext * d23_knext * disc_fact
                #
                Beta_kplus1[18][transition_counter] = v3_knext * u1_knext * disc_fact
                #
                Beta_kplus1[19][transition_counter] = v3_knext * u2_knext * disc_fact
                #
                Beta_kplus1[20][transition_counter] = v3_knext * u3_knext * disc_fact
                                #
                Beta_kplus1[21][transition_counter] = d12_knext * d12_knext * disc_fact
                #
                Beta_kplus1[22][transition_counter] = d12_knext * d23_knext * disc_fact
                #
                Beta_kplus1[23][transition_counter] = d12_knext * u1_knext * disc_fact
                #
                Beta_kplus1[24][transition_counter] = d12_knext * u2_knext * disc_fact
                #
                Beta_kplus1[25][transition_counter] = d12_knext * u3_knext * disc_fact
                #
                Beta_kplus1[26][transition_counter] = d23_knext * d23_knext * disc_fact
                #
                Beta_kplus1[27][transition_counter] = d23_knext * u1_knext * disc_fact
                                #
                Beta_kplus1[28][transition_counter] = d23_knext * u2_knext * disc_fact
                #
                Beta_kplus1[29][transition_counter] = d23_knext * u3_knext * disc_fact
                #
                Beta_kplus1[30][transition_counter] = u1_knext * u1_knext * disc_fact
                #
                Beta_kplus1[31][transition_counter] = u1_knext * u2_knext * disc_fact
                #
                Beta_kplus1[32][transition_counter] = u1_knext * u3_knext * disc_fact
                #
                Beta_kplus1[33][transition_counter] = u2_knext * u2_knext * disc_fact
                #
                Beta_kplus1[34][transition_counter] = u2_knext * u3_knext * disc_fact
                #
                Beta_kplus1[35][transition_counter] = u3_knext * u3_knext * disc_fact
                #
                Beta_kplus1[36][transition_counter] = 1 * disc_fact
                
                #################################################
                #populate Ck
                C_k[transition_counter] = trans_cost
                
                transition_counter +=1
                
        if transition_counter != N:
            print('Error!! Matrices do not match')
        else:
            print('All Done!')
            
        # Convert to matrices of the form Ax=b
        A_mat = np.matmul(Beta_k,(Beta_k-Beta_kplus1).transpose())
        b_mat = np.matmul(Beta_k,C_k)
        
        print(f'Rejected episodes: {rejected_episodes}')
        return A_mat,b_mat

def GradDescent(A,b,Max_iterations = 100000, tolerance = 0.0001):
        #Takes in a matrix and a vector (formulated above) and uses gradient descent to solve for theta since in an Ax-B format


        # epsilon is the step length

        # Find the epsilon that makes it stable by cycling through smaller and smaller until the eigenvalues are all less than 1
        # In practice the eigenvalues will always be less than one so make sure they're less than 1.0000001

        Theta_size = b.shape[0]
        epsil = 0.1
        eye_size = np.identity(Theta_size)

        while True:
            grad_1=eye_size-epsil*A
            w,v = np.linalg.eig(grad_1)

            if max(abs(w)) < 1.0000001:
                break
            else:
                epsil = epsil/10

        print(f'stable epsilon found = {epsil}')
        #Initialise Theta0
        Theta0 = np.ones((Theta_size,1))
        #pdb.set_trace()

        # Define convergence tolerance
        ################################
        delta = tolerance
        ################################
        ten_percent = Max_iterations/10
        timer_done = False
        start_time = time.time()

        #Update Theta for every time step until convergence
        for t_iter in range(Max_iterations):
            
            grad = (np.matmul(A,Theta0)-b)
            
            if t_iter%ten_percent == 0 and t_iter !=0:
                print(f'...running Gradient Descent {100*t_iter/Max_iterations}% completed')
                print(f'Current maximum error = {max(abs(grad))}')
                
                if not timer_done:
                    elapsed = time.time() - start_time
                    Forecast_time_remaining = 9*elapsed
                    print(f'maximum remaining time: {round(Forecast_time_remaining/60)} minutes')
                    timer_done = True
            

            if max(abs(grad))<delta:
                break
            else:
                
                Theta0 -= epsil * grad
                
                if t_iter == Max_iterations-1:
                    print('Max iterations exceeded, check convergence!')
                    error = np.matmul(A,Theta0)-b
                    maxerror = MatrixMax(error)
                    print(f'Max Error: {maxerror}')
                    
                    
                # if we get halfway and it is pretty close, use it and carry on
                # if it's clearly diverging, use it but say it's rubbish
                if t_iter == math.floor(Max_iterations/2):
                    print('Halfway there')
                    
                    #if max(abs(grad))< (delta*20):
                    #    print(f'Within {delta*20} so breaking')
                    #    break
                    if max(abs(grad)) > 10e10:
                        print('Biggest error is >10e10. It will not converge. Breaking..')
                        break
                    else:
                        print('Error is decreasing but not quite there yet, carry on')
                        
                    
                    
                    
        print('All done!')

        return Theta0,t_iter


def PolicyMaximiser(Theta):
    #Given an estimation, Theta of the Q function, this will minimise over the action to space to find the optimal gains
    # It requires solving 3 simultaneous equations

    #The new K matrix ( for u = Kx) is found solving this:

    Matr1 = -np.array([[2*Theta[30][0],Theta[31][0],Theta[32][0]],
                     [Theta[31][0], 2*Theta[33][0], Theta[34][0]],
                     [Theta[32][0], Theta[34][0],2*Theta[35][0]]])
    Matr2 = np.array([[Theta[5][0],Theta[23][0],Theta[12][0],Theta[27][0],Theta[18][0]],
                     [Theta[6][0],Theta[24][0],Theta[13][0],Theta[28][0],Theta[19][0]],
                     [Theta[7][0],Theta[25][0],Theta[14][0],Theta[29][0],Theta[20][0]]])

    new_K = np.linalg.solve(Matr1, Matr2)

    #pdb.set_trace()

    #array of 3x5
    return new_K

############################################
## Run controller Improvement
############################################

In [ ]:
K_initial = np.array([[-.2, -.01, 0,0, 0],
                   [0, 0.01, -.2, -0.01, 0],
                   [0, 0, 0,0.01,-.2]])

In [ ]:
# Variables to input:

# Initial controller
K = K_initial######

# Number of updates, episodes and transitions:
N_updates = 20 # leave as 20 unless testing #######
N_episodes = 10000 #VARY
N_transitions = 500 #leave as 500

# Set global variables
noisy_controller = True
model_variance = .2
controller_variance = 0.25

LoK = [K] #######
Thet = [] #######
iteratation_count = 1

for t in range(N_updates):

    #pdb.set_trace()
    A0,B0 = LSOptimiser(K,N_episodes,N_transitions)
    Thet0,n0 = GradDescent(A0,B0,10000000,tolerance=0.0001)
    K = PolicyMaximiser(Thet0)
    LoK.append(K) ####### 
    Thet.append(Thet0) ########
    print(f'\n Iteration {iteratation_count} done.\n K found is: {K}')
    iteratation_count +=1

############################################
## Test controllers
############################################

In [ ]:
n_tests = 100
K_list = LoK ########

List_of_IC = []
for i in range(n_tests):
    np.random.seed(i)
    List_of_IC.append([-15 + 20*np.random.rand(),-5 + 25*np.random.rand(),-15 + 20*np.random.rand(),-5 + 25*np.random.rand(),-15 + 20*np.random.rand()])       

noisy_controller=False
model_variance = 0
controller_variance = 0.25  # CONTROLLER NOISE IS FALSE ANYWAY

discarded_eps_list = [] ######
LoC = []
LoC_nodiscards = [] ########
done = 0
for K_ in K_list:

    discards = 0
    Av_cost_nodiscards = 0
    Av_cost = 0
    for test_IC in List_of_IC:  #for every IC in the list, add the cost to the variable av_cost
        #pdb.set_trace()
        vehicle1,vehicle2,vehicle3 = Initialise(test_IC)
        #pdb.set_trace()
        v1,v2,v3,d12,d23,Co,u1,u2,u3,flag = RunEpisode(500,vehicle1,vehicle2,vehicle3,K_)
        
        #add to this cost tally even if flagged
        Av_cost_nodiscards += GetDiscountedCost(Co)
        if flag ==True:
            discards +=1
        else:
            # only add to this tally if flag is False
            Av_cost += GetDiscountedCost(Co)
            
        #pdb.set_trace()
        
    Av_cost = Av_cost/(n_tests-discards) # divide by only those that contributed
    Av_cost_nodiscards = Av_cost_nodiscards/(n_tests)
    LoC_nodiscards.append(Av_cost_nodiscards) ######
    LoC.append(Av_cost)
    discarded_eps_list.append(discards) #######
    done+=1
    print(f'{done} controllers done')
